In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%matplotlib inline

In [ ]:
eco_df = pd.read_excel('../data/TEIS-NSS Project Data 10-2022.xlsx',sheet_name=('ECO with Exit21-22'))
elig_df = pd.read_excel('../data/TEIS-NSS Project Data 10-2022.xlsx',sheet_name=('Elig Timeline Rpt 2018-2022'))

In [ ]:
eco_df.head()

In [ ]:
eco_df.drop(eco_df.columns.difference(['CHILD_ID', 'Ent SOCIAL_SCALE', 'Ent KNOWLEDGE_SCALE', 'Ent APPROPRIATE_ACTION_SCALE', 'Exit SOCIAL_SCALE', 'Exit KNOWLEDGE_SCALE', 'Exit APPROPRIATE_ACTION_SCALE', 'OC1 - a', 'OC1 - b', 'OC1 - c', 'OC1 - d', 'OC1 - e', 'OC1 Data Check', 'OC2 - a', 'OC2 - b', 'OC2 - c', 'OC2 - d', 'OC2 - e', 'OC2 Data Check', 'OC3 - a', 'OC3 - b', 'OC3 - c', 'OC3 - d', 'OC3 - e', 'OC3 Data Check']), axis=1, inplace=True)
display(eco_df)

In [ ]:
elig_df.head()

In [ ]:
elig_df.drop(elig_df.columns.difference(['Child ID', 'Init. Elig. Category']), axis=1, inplace=True)
print(elig_df)

In [ ]:
eco_df = eco_df[eco_df['OC2 Data Check'] != 0]
display(eco_df)

In [ ]:
eco_df = eco_df[eco_df['OC2 Data Check'].notna()]
display(eco_df)

In [ ]:
eco_df.rename({'CHILD_ID':'child_id', 'Ent SOCIAL_SCALE':'ent_social', 'Ent KNOWLEDGE_SCALE':'ent_knowledge', 'Ent APPROPRIATE_ACTION_SCALE':'ent_app_action', 'Exit SOCIAL_SCALE':'exit_social', 'Exit KNOWLEDGE_SCALE':'exit_knowledge', 'Exit APPROPRIATE_ACTION_SCALE':'exit_app_action', 'OC1 - a':'oc1a', 'OC1 - b':'oc1b', 'OC1 - c':'oc1c', 'OC1 - d':'oc1d', 'OC1 - e':'oc1e', 'OC1 Data Check':'oc1_data_check', 'OC2 - a':'oc2a', 'OC2 - b':'oc2b', 'OC2 - c':'oc2c', 'OC2 - d':'oc2d', 'OC2 - e':'oc2e', 'OC2 Data Check':'oc2_data_check', 'OC3 - a':'oc3a', 'OC3 - b':'oc3b', 'OC3 - c':'oc3c', 'OC3 - d':'oc3d', 'OC3 - e':'oc3e', 'OC3 Data Check':'oc3_data_check'}, axis=1, inplace=True)

In [ ]:
elig_df.rename({'Child ID':'child_id', 'Init. Elig. Category':'init_elig_category'}, axis=1, inplace=True)

In [ ]:
eco_elig_combined = pd.merge(eco_df,elig_df, on='child_id')
eco_elig_combined.info()

In [ ]:
diff_by_elig = eco_elig_combined[['ent_social', 'ent_knowledge', 'ent_app_action', 'exit_social', 'exit_knowledge', 'exit_app_action', 'init_elig_category']]
diff_by_elig['social_diff'] = diff_by_elig['exit_social'] - diff_by_elig['ent_social']
diff_by_elig['knowledge_diff'] = diff_by_elig['exit_knowledge'] - diff_by_elig['ent_knowledge']
diff_by_elig['app_action_diff'] = diff_by_elig['exit_app_action'] - diff_by_elig['ent_app_action']
diff_by_elig = diff_by_elig[['social_diff', 'knowledge_diff', 'app_action_diff', 'init_elig_category']]
display(diff_by_elig)

## Sum improvement on all fronts based on elig category

In [ ]:
sum_improvements = diff_by_elig.groupby('init_elig_category').sum()
display(sum_improvements)